# Preparing the dataset

We build a dataset using tfrecords and webp format.

In [ ]:
# install dependencies
#!pip install img2dataset tensorflow tensorflow_io wandb

## Getting URL list

We use the [Conceptual Captions](https://ai.google.com/research/ConceptualCaptions/).

In [ ]:
# Download Conceptual Captions
!wget https://storage.googleapis.com/gcc-data/Validation/GCC-1.1.0-Validation.tsv -O GCC-valid.tsv
!wget https://storage.googleapis.com/gcc-data/Train/GCC-training.tsv -O GCC-train.tsv

We format input files to keep only url's.

In [ ]:
import pandas as pd

In [ ]:
# clean up the files and keep only url
for f, name in zip(
    ["GCC-train.tsv", "GCC-valid.tsv"],
    ["train", "valid"],
):
    df = pd.read_csv(f, sep="\t", names=["caption", "url"])
    df.to_parquet(f"{name}.parquet")

Datasets may be a bit large so we reduce their size.

In [ ]:
for path, max_items in [("train.parquet", 500_000), ("valid.parquet", 10_000)]:
    df = pd.read_parquet(path)
    print(f"{path}: keeping {max_items} / {len(df)}")
    df = df[:max_items]
    df.to_parquet(path)

## Download images

In [ ]:
!mkdir cc3m

In [ ]:
# parameters for validation set
input_file = "valid.parquet"
output_folder = "cc3m/valid"
input_format = "parquet"
caption_col = "caption"
image_size = 256
processes_count = 80
thread_count = 16
encode_quality = 100
encode_format = "webp"
number_sample_per_shard = 1000
min_image_size = 128

In [ ]:
!img2dataset \
  --url_list $input_file \
  --image_size $image_size \
  --output_folder $output_folder \
  --input_format $input_format \
  --caption_col $caption_col \
  --processes_count $processes_count \
  --thread_count $thread_count \
  --resize_mode center_crop \
  --encode_quality $encode_quality \
  --encode_format $encode_format \
  --output_format tfrecord \
  --number_sample_per_shard $number_sample_per_shard \
  --extract_exif false \
  --min_image_size $min_image_size \
  --enable_wandb

In [ ]:
# update relevant parameters for train set
input_file = "train.parquet"
output_folder = "cc3m/train"

In [ ]:
!img2dataset \
  --url_list $input_file \
  --image_size $image_size \
  --output_folder $output_folder \
  --input_format $input_format \
  --caption_col $caption_col \
  --processes_count $processes_count \
  --thread_count $thread_count \
  --resize_mode center_crop \
  --encode_quality $encode_quality \
  --encode_format $encode_format \
  --output_format tfrecord \
  --number_sample_per_shard $number_sample_per_shard \
  --extract_exif false \
  --min_image_size $min_image_size \
  --enable_wandb

## Dataloader

Files have been saved as tfrecords

In [ ]:
from clip_jax.data import Dataset, logits_to_image
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm

## Optional: Calculate mean and std of the dataset

We calculate the statistics on the validation set to use a center crop instead of random crop.

In [ ]:
dataset = Dataset(
    valid_folder="cc3m/train",
    valid_batch_size=1000,
    image_size=224,
)

We parallelize the calculation of mean and std because we are efficient people.

In [ ]:
def get_mean_std(images, captions):
    images = tf.cast(images, tf.float64)
    mean = tf.reduce_mean(images, axis=(0, 2, 3))
    std = tf.math.reduce_std(images, axis=(0, 2, 3))
    return mean, std

In [ ]:
ds_mean_std = dataset.valid.map(
    get_mean_std, num_parallel_calls=tf.data.experimental.AUTOTUNE
)

In [ ]:
means = []
stds = []

for batch in tqdm(ds_mean_std):
    mean, std = batch
    means.append(mean)
    stds.append(std)

In [ ]:
# get the global mean and std
mean = tf.stack(means, axis=0)
std = tf.stack(stds, axis=0)
mean = tf.reduce_mean(mean, axis=0)
std = tf.math.sqrt(tf.reduce_sum(tf.math.square(std) / len(stds), axis=0))

mean, std

## Dataloader

In [ ]:
dataset = Dataset(
    train_folder="cc3m/train",
    valid_folder="cc3m/train",
    train_batch_size=10,
    valid_batch_size=10,
    image_size=224,
    mean=[0.5, 0.5, 0.5],
    std=[0.5, 0.5, 0.5],
)

In [ ]:
sample_batch = next(iter(dataset.train.as_numpy_iterator()))
sample_batch

In [ ]:
images = sample_batch[0]

In [ ]:
# visualize the batch
plt.figure(figsize=(10, 10))
for i in range(9):
    img = logits_to_image(
        images[i], mean=dataset.mean, std=dataset.std, format=dataset.format
    )
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(img)

Let's check captions.

In [ ]:
from transformers import CLIPTokenizerFast

In [ ]:
CLIP_REPO = "openai/clip-vit-large-patch14"
tokenizer = CLIPTokenizerFast.from_pretrained(CLIP_REPO)

In [ ]:
captions = sample_batch[1]
captions[:5]

In [ ]:
# tokenize
captions = [caption.decode("utf-8") for caption in captions]
txt_inputs = tokenizer(
    captions, padding="max_length", truncation=True, return_tensors="np"
)
txt_inputs

## Run CLIP on data

In [ ]:
from transformers import FlaxCLIPModel

In [ ]:
model = FlaxCLIPModel.from_pretrained(CLIP_REPO)

In [ ]:
inputs = {"pixel_values": images, **txt_inputs}

In [ ]:
outputs = model(**inputs)[0]